# Match contours


## Setup


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from loguru import logger as lg
from rich import get_console
from rich import print as rprint
from rich.console import Console

# some magic to make rich work in jupyter
# https://github.com/Textualize/rich/issues/3483
# enable it for every cell output with %load_ext rich
console: Console = get_console()
console.is_jupyter = False

In [ ]:
from snap_fit.config.aruco.aruco_board_config import ArucoBoardConfig
from snap_fit.config.aruco.aruco_detector_config import ArucoDetectorConfig
from snap_fit.config.types import EDGE_ENDS_TO_CORNER
from snap_fit.config.types import EdgePos
from snap_fit.image.process import find_contours
from snap_fit.image.process import find_corners
from snap_fit.image.segment_matcher import SegmentMatcher
from snap_fit.image.utils import draw_contour
from snap_fit.image.utils import draw_corners
from snap_fit.image.utils import show_image_mpl
from snap_fit.params.snap_fit_params import get_snap_fit_paths
from snap_fit.puzzle.piece import Piece
from snap_fit.puzzle.sheet import Sheet
from snap_fit.puzzle.sheet_aruco import SheetAruco
from snap_fit.puzzle.sheet_manager import SheetManager


In [ ]:
sf_paths = get_snap_fit_paths()
sample_fol = sf_paths.sample_img_fol

In [ ]:
img_fn = "back_03.jpg"
# img_fn = "back_04.jpg"
img_fp = sample_fol / img_fn
img_fp

In [ ]:
sheet = Sheet(img_fp)
lg.info(f"found {len(sheet.pieces)} pieces")

In [ ]:
show_image_mpl(sheet.img_orig)

In [ ]:
piece = sheet.pieces[5]

In [ ]:
img_contour = draw_contour(piece.img_bw, piece.contour_loc, color=127)
corners = list(piece.corners.values())
img_corners = draw_corners(img_contour, corners, color=190)
show_image_mpl(img_corners)

In [ ]:
piece.corners

## Split contour


### Find the corners in the contour


In [ ]:
for corner_name in piece.corners:
    corner_coords = piece.corners[corner_name]
    cont_corner_idxs = piece.contour.corner_idxs[corner_name]
    cont_corner_coords = piece.contour.corner_coords[corner_name]
    lg.debug(
        f"{corner_name}: {corner_coords} -> {cont_corner_idxs} -> {cont_corner_coords}"
    )

### Split the contour in four segments


In [ ]:
for edge_name, edge_ends in EDGE_ENDS_TO_CORNER.items():
    start_idx = piece.contour.corner_idxs[edge_ends[0]]
    end_idx = piece.contour.corner_idxs[edge_ends[1]]
    ends_coords = piece.contour.segments[edge_name].coords
    lg.debug(f"{edge_name}: {start_idx} -> {end_idx} ({ends_coords})")

In [ ]:
tot_len = 0
for edge_name in EDGE_ENDS_TO_CORNER:
    segment = piece.contour.segments[edge_name]
    points = segment.points
    lg.debug(f"{edge_name}: {len(points)}")
    tot_len += len(points)
lg.debug(f"total length: {tot_len}")
lg.debug(f"total contour length: {len(piece.contour_loc)}")

In [ ]:
img_contour_seg = piece.img_bw.copy() // 10
for ei, edge_name in enumerate(EdgePos):
    segment = piece.contour.segments[edge_name]
    points = segment.points
    img_contour_seg = draw_contour(img_contour_seg, points, color=120 + ei * 40)
show_image_mpl(img_contour_seg)

## Match segments


### Translate the segments


In [ ]:
p1_index = 2
p2_index = 5

# p1 = sheet.pieces[3]
# p2 = sheet.pieces[4]

s1_type = EdgePos.RIGHT
s2_type = EdgePos.TOP

p1 = sheet.pieces[p1_index]
p2 = sheet.pieces[p2_index]

seg1 = p1.contour.segments[s1_type]
seg2 = p2.contour.segments[s2_type]

In [ ]:
img_contour_seg = p1.img_bw.copy() // 10
for ei, edge_name in enumerate(EdgePos):
    segment = p1.contour.segments[edge_name]
    points = segment.points
    draw_contour(img_contour_seg, points, color=120 + ei * 40)
show_image_mpl(img_contour_seg, figsize=(5, 5))

img_contour_seg = p2.img_bw.copy() // 10
for ei, edge_name in enumerate(EdgePos):
    segment = p2.contour.segments[edge_name]
    points = segment.points
    draw_contour(img_contour_seg, points, color=120 + ei * 40)
show_image_mpl(img_contour_seg, figsize=(5, 5))

### Match the segments


In [ ]:
# should we rescale the dist on the distance between the ends?
# should we take the average of the distances between the points?

#### With class


In [ ]:
seg_match = SegmentMatcher(seg1, seg2)
seg_match.compute_similarity()

In [ ]:
# draw the internal segments
p1_img = p1.img_bw.copy() // 10
draw_contour(p1_img, seg_match.s1.points, color=120)
show_image_mpl(p1_img, figsize=(5, 5))

In [ ]:
# draw the internal segments
p2_img = p2.img_bw.copy() // 10
draw_contour(p2_img, seg_match.s2.points, color=120)
draw_contour(p2_img, seg_match.s1_points_transformed, color=120)
show_image_mpl(p2_img, figsize=(5, 5))

## Iterate all the pairs of pieces


In [ ]:
all_pieces = sheet.pieces
len(all_pieces)

In [ ]:
p1 = all_pieces[3]
p2 = all_pieces[4]

In [ ]:
def match_pieces(p1: Piece, p2: Piece) -> dict:
    """Match all edges between two pieces and return a dict of similarities.

    Keys of the dict are tuples of ((p1_name, p1_id, e1_type),
    (p2_name, p2_id, e2_type)) where e1_type and e2_type are edge positions.
    Values are the similarity scores (float).
    """
    distances_pieces = {}

    for e1_type in EdgePos:
        for e2_type in EdgePos:
            seg1 = p1.segments[e1_type]
            seg2 = p2.segments[e2_type]
            seg_match = SegmentMatcher(seg1, seg2)
            sim = seg_match.compute_similarity()
            pe1 = (p1.name, p1.piece_id, e1_type)
            pe2 = (p2.name, p2.piece_id, e2_type)
            distances_pieces[(pe1, pe2)] = float(sim)

    return distances_pieces

In [ ]:
distances_pieces = match_pieces(p1, p2)
distances_pieces

In [ ]:
def match_all_pieces(all_pieces: list[Piece]) -> list:
    """Compute distances between all segments and all pieces."""
    distances_all = {}

    for i1, p1 in enumerate(all_pieces):
        for i2, p2 in enumerate(all_pieces[i1 + 1 :]):
            if i1 == i2:
                continue
            distances_pieces = match_pieces(p1, p2)
            distances_all.update(distances_pieces)
    sort_dist = sorted(distances_all.items(), key=lambda x: x[1])
    return sort_dist


sort_dist = match_all_pieces(all_pieces)
sort_dist[:15]

In [ ]:
# best match
best_match = sort_dist[0]
best_match

In [ ]:
pair = best_match[0]
pair

In [ ]:
p1_res = pair[0]
p1_index = p1_res[1]
s1_type = p1_res[2]

p2_res = pair[1]
p2_index = p2_res[1]
s2_type = p2_res[2]

p1 = sheet.pieces[p1_index]
p2 = sheet.pieces[p2_index]

seg1 = p1.contour.segments[s1_type]
seg2 = p2.contour.segments[s2_type]

# create a segment matcher so that s1 points get transformed
seg_match = SegmentMatcher(seg1, seg2)

# draw the internal segments
p2_img = p2.img_bw.copy() // 10
draw_contour(p2_img, seg_match.s2.points, color=120)
draw_contour(p2_img, seg_match.s1_points_transformed, color=120)
show_image_mpl(p2_img, figsize=(5, 5))

## Load new data


In [ ]:
# 1. Configure ArUco Board and Detector
# Using defaults which match the printed board used for 'data/oca'
board_config = ArucoBoardConfig(markers_x=5, markers_y=7)
detector_config = ArucoDetectorConfig(board=board_config)

# 2. Initialize SheetAruco helper
# crop_margin is automatically calculated from the detector configuration
sheet_aruco = SheetAruco(detector_config)

# 3. Define the loader function
# SheetAruco.load_sheet handles loading, rectification, and Sheet creation
aruco_loader = sheet_aruco.load_sheet

# 4. define base folder
paths = get_snap_fit_paths()
data_dir = paths.data_fol / "oca"
lg.info(f"Loading data from {data_dir}")

# 5. instantiate manager and load
manager = SheetManager()
manager.add_sheets(folder_path=data_dir, pattern="*.jpg", loader_func=aruco_loader)

# Verify Sheets
sheets = manager.get_sheets_ls()
print(f"Managed Sheets: {len(sheets)}")
for sheet in sheets:
    print(f" - {sheet.img_fp.name}: {len(sheet.pieces)} pieces")

# Verify Pieces
pieces = manager.get_pieces_ls()
print(f"Total Pieces: {len(pieces)}")


In [ ]:
# match them all
matches_oca = match_all_pieces(pieces)


In [ ]:
def get_piece(name: str, index: int, pieces: list[Piece]) -> Piece:
    """Get a piece given name and index."""
    for p in pieces:
        if p.name == name and p.piece_id == index:
            return p
    msg = "piece not found"
    raise ValueError(msg)

In [ ]:
best_match_oca = matches_oca[20]
rprint(best_match_oca)

pair_bm_oca = best_match_oca[0]
# rprint(pair_bm_oca)

p1_res = pair_bm_oca[0]
p1_name = p1_res[0]
p1_index = p1_res[1]
s1_type = p1_res[2]
# p1 = pieces[p1_index]
p1 = get_piece(p1_name, p1_index, pieces)
seg1 = p1.contour.segments[s1_type]

p2_res = pair_bm_oca[1]
p2_name = p2_res[0]
p2_index = p2_res[1]
s2_type = p2_res[2]
p2 = get_piece(p2_name, p2_index, pieces)
seg2 = p2.contour.segments[s2_type]
rprint(f"using {p1.name} and {p2.name}")

# create a segment matcher so that s1 points get transformed
seg_match = SegmentMatcher(seg1, seg2)
sim = seg_match.compute_similarity()
rprint(f"got {sim=:.2f}")

# draw the internal segments
p2_img = p2.img_bw.copy() // 10
draw_contour(p2_img, seg_match.s2.points, color=120)
draw_contour(
    p2_img,
    seg_match.s1_points_transformed + 20,
    color=180,
)
show_image_mpl(p2_img, figsize=(5, 5))